In [ ]:
def read_uint64_pointer(val, count):
    # val: gdb.Value pointing to uint64_t *
    # count: number of elements to read
    inf = gdb.selected_inferior()
    address = int(val)
    length = count * 8  # 8 bytes per uint64_t
    mem = inf.read_memory(address, length)
    # Convert bytes to list of uint64_t (big-endian)
    result = [int.from_bytes(mem[i*8:(i+1)*8], 'big') for i in range(count)]
    return result

In [2]:
def read_bits(words, bit_offset, num_bits):
    # words: list of 64-bit integers
    # bit_offset: starting bit position (0 = MSB of words[0])
    # num_bits: number of bits to read
    result = 0
    for i in range(num_bits):
        pos = bit_offset + i
        word_idx = pos // 64
        bit_idx = pos % 64
        # MSB is bit 0, so shift by (63 - bit_idx)
        bit = (words[word_idx] >> (63 - bit_idx)) & 1
        result = (result << 1) | bit
    return result

In [3]:
# Test read_bits function
words = [0xF0F0F0F0F0F0F0F0, 0x123456789ABCDEF0]
bit_offset = 60
num_bits = 8
result = read_bits(words, bit_offset, num_bits)
print(f"Bits from offset {bit_offset} (next {num_bits} bits): {bin(result)} ({result})")

# Test reading across word boundary
bit_offset2 = 62
num_bits2 = 6
result2 = read_bits(words, bit_offset2, num_bits2)
print(f"Bits from offset {bit_offset2} (next {num_bits2} bits): {bin(result2)} ({result2})")

Bits from offset 60 (next 8 bits): 0b1 (1)
Bits from offset 62 (next 6 bits): 0b1 (1)


In [5]:
# Read 12 bits from offset 56
bit_offset3 = 56
num_bits3 = 12
result3 = read_bits(words, bit_offset3, num_bits3)
print(f"Bits from offset {bit_offset3} (next {num_bits3} bits): {bin(result3)} ({result3}), hex: {hex(result3)}")

Bits from offset 56 (next 12 bits): 0b111100000001 (3841), hex: 0xf01


## How to call a member function of a C++ object with gdb.Value in GDB Python
To call a member function of an object using a `gdb.Value`, use the `gdb.parse_and_eval` function with the appropriate C++ expression as a string.

**Example:**
```python
# If obj is a gdb.Value representing an object (not a pointer):
result = gdb.parse_and_eval(f'({obj.type.pointer()}){int(obj.address)}->foo()')

# If obj is a pointer already:
result = gdb.parse_and_eval(f'{obj}.foo()')

# Or using the object's name if available:
result = gdb.parse_and_eval('obj.foo()')
```
This will evaluate the member function and return a new `gdb.Value` with the result.

## Accessing static constexpr members of a class in GDB Python
To access a `static constexpr` member of a class, use `gdb.parse_and_eval` with the fully qualified name:
```python
value = gdb.parse_and_eval('ClassName::constexpr_member')
```
Replace `ClassName` and `constexpr_member` with your actual class and member names.
This works for both `static constexpr` and `static const` members, as long as they are accessible in the debug info. No object instance is needed.

In [ ]:
import traceback

try:
    # ... your code ...
except Exception as e:
    tb = traceback.extract_tb(e.__traceback__)
    last_entry = tb[-1]
    print(f"Exception at {last_entry.filename}, line {last_entry.lineno}: {last_entry.line}")